In [1]:
import pandas as pd
from sqlalchemy import create_engine

### Store CSV into DataFrame

In [4]:
csv_file_beers = "Resources/beers.csv"
beers_df = pd.read_csv(csv_file_beers)
beers_df.head()

,Unnamed: 0,abv,ibu,id,name,style,brewery_id,ounces
0,0,0.050,NaN,1436,Pub Beer,American Pale Lager,408,12.0
1,1,0.066,NaN,2265,Devil's Cup,American Pale Ale (APA),177,12.0
2,2,0.071,NaN,2264,Rise of the Phoenix,American IPA,177,12.0
3,3,0.090,NaN,2263,Sinister,American Double / Imperial IPA,177,12.0
4,4,0.075,NaN,2262,Sex and Candy,American IPA,177,12.0


In [5]:
newbeer_df = beers_df[['abv', 'id', 'name','style', 'brewery_id', 'ounces']].copy()

newbeer_df.head()

,abv,id,name,style,brewery_id,ounces
0,0.050,1436,Pub Beer,American Pale Lager,408,12.0
1,0.066,2265,Devil's Cup,American Pale Ale (APA),177,12.0
2,0.071,2264,Rise of the Phoenix,American IPA,177,12.0
3,0.090,2263,Sinister,American Double / Imperial IPA,177,12.0
4,0.075,2262,Sex and Candy,American IPA,177,12.0


In [7]:
csv_file_breweries = "Resources/breweries.csv"
breweries_df = pd.read_csv(csv_file_breweries)
breweries_df.head()

,Unnamed: 0,name,city,state
0,0,NorthGate Brewing,Minneapolis,MN
1,1,Against the Grain Brewery,Louisville,KY
2,2,Jack's Abby Craft Lagers,Framingham,MA
3,3,Mike Hess Brewing Company,San Diego,CA
4,4,Fort Point Beer Company,San Francisco,CA


### Create new data with select columns

In [8]:
newbrew_df = breweries_df[['Unnamed: 0','name', 'city', 'state']].copy()
newbrew_df = newbrew_df.rename(columns={'name':'brewery_name','Unnamed: 0':'brewery_id'})
newbrew_df.set_index('brewery_id')

,brewery_name,city,state
brewery_id,,,
0,NorthGate Brewing,Minneapolis,MN
1,Against the Grain Brewery,Louisville,KY
2,Jack's Abby Craft Lagers,Framingham,MA
3,Mike Hess Brewing Company,San Diego,CA
4,Fort Point Beer Company,San Francisco,CA
...,...,...,...
553,Covington Brewhouse,Covington,LA
554,Dave's Brewfarm,Wilson,WI
555,Ukiah Brewing Company,Ukiah,CA


### Connect to local database

In [17]:
rds_connection_string = "postgres:postgres@localhost:5432/Project_2"
engine = create_engine(f'postgresql://{rds_connection_string}')

### Check for tables

In [18]:
engine.table_names()

[]

In [19]:
newbrew_df.to_csv(r"Resources/breweries2.csv")
newbeer_df.to_csv(r"Resources/beers2.csv")

### Use pandas to load csv converted DataFrame into database

In [20]:
newbrew_df.to_sql(name='breweries', con=engine, if_exists='append', index=False)

In [21]:
newbeer_df.to_sql(name='beers', con=engine, if_exists='append', index=True)

### Confirm data has been added by querying the customer_name table
* NOTE: can also check using pgAdmin

In [22]:
pd.read_sql_query('select * from beers', con=engine).head()

,index,abv,id,name,style,brewery_id,ounces
0,0,0.050,1436,Pub Beer,American Pale Lager,408,12.0
1,1,0.066,2265,Devil's Cup,American Pale Ale (APA),177,12.0
2,2,0.071,2264,Rise of the Phoenix,American IPA,177,12.0
3,3,0.090,2263,Sinister,American Double / Imperial IPA,177,12.0
4,4,0.075,2262,Sex and Candy,American IPA,177,12.0


### Confirm data has been added by querying the customer_location table

In [72]:
pd.read_sql_query('select * from breweries', con=engine).head()

,brewery_id,brewery_name,city,state
0,0,NorthGate Brewing,Minneapolis,MN
1,1,Against the Grain Brewery,Louisville,KY
2,2,Jack's Abby Craft Lagers,Framingham,MA
3,3,Mike Hess Brewing Company,San Diego,CA
4,4,Fort Point Beer Company,San Francisco,CA
